In [1]:
import ee
import geemap
import pandas as pd
import numpy as np
import os
import geopandas as gpd

In [2]:
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()


In [3]:
os.chdir('/Users/najah/work/internships/meghna//LT05_L1TP_145044_20100428_20161016_01_T1')

In [4]:
## adding points from local machine
Map = geemap.Map()
points_shp = './145044_20100428_roi/145044_20100428_500points.random_points2/145044_20100428_500points.shp'
points = geemap.shp_to_ee(points_shp)
Map.addLayer(   points, {'color':'red'}, '500points')
Map.centerObject(points,8)
Map
 

Map(center=[23.101347181130787, 78.0196924881727], controls=(WidgetControl(options=['position', 'transparent_b…

In [5]:
## adding the landsat image

image = ee.Image('LANDSAT/LT05/C01/T1_TOA/LT05_145044_20100428') \
            .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])

Map.centerObject(image)

rgbVis = {
  'min':0,
  'max' :0.4,
  'bands' : ['B5', 'B4', 'B3'],
  'gamma' : 1.2
}

Map.addLayer(image, rgbVis, 'original')
Map

Map(center=[23.101347181130787, 78.0196924881727], controls=(WidgetControl(options=['position', 'transparent_b…

In [6]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5',  'B7']

# This property of the table stores the land cover labels.
label = 'class'

# Overlay the points on the imagery to get training.
# sample = image.select(bands).sampleRegions(
#     **{'collection': points, 'properties': [label], 'scale': 30}
# )

training = image.select(bands).sampleRegions(**{
  'collection' : points,
  'properties' : [label],
  'scale': 30
})

## building the classifer
Args:
  decisionProcedure: The decision procedure to use for
      classification. Either 'Voting' or 'Margin'. Not
      used for regression.
      
  svmType: The SVM type. One of `C_SVC`, `NU_SVC`, `ONE_CLASS`,
      `EPSILON_SVR` or `NU_SVR`.
      
  kernelType: The kernel type. One of LINEAR (u′×v), POLY
      ((γ×u′×v + coef₀)ᵈᵉᵍʳᵉᵉ), RBF (exp(-γ×|u-v|²)) or
      SIGMOID (tanh(γ×u′×v + coef₀)).
      
  shrinking: Whether to use shrinking heuristics.
  degree: The degree of polynomial. Valid for POLY kernels.
  
  gamma: The gamma value in the kernel function. Defaults to the
      reciprocal of the number of features. Valid for POLY, RBF
      and SIGMOID kernels.
      
  coef0: The coef₀ value in the kernel function. Defaults to 0.
      Valid for POLY and SIGMOID kernels.
      
  cost: The cost (C) parameter. Defaults to 1. Only valid for
      C-SVC, epsilon-SVR, and nu-SVR.
      
  nu: The nu parameter. Defaults to 0.5. Only valid for nu-SVC, one-
      class SVM, and nu-SVR.
      
  terminationEpsilon: The termination criterion
      tolerance (e). Defaults to 0.001. Only valid
      for epsilon-SVR.
      
  lossEpsilon: The epsilon in the loss function (p).
      Defaults to 0.1. Only valid for epsilon-SVR.
  oneClass: The class of the training data on which to train in
      a one-class SVM.  Defaults to 0. Only valid for one-class
      SVM.  Possible values are 0 and 1.  The classifier output
      is binary (0/1) and will match this class value for the
      data determined to be in the class.is binary (0/1) and will match this class value for the data determined to be in the class.mm
      _

classifier = ee.Classifier.smileRandomForest( numberOfTrees = 500, variablesPerSplit = 5).train(training, label, bands)

In [10]:
classifier  =ee.Classifier.libsvm( kernelType = 'RBF',gamma= 0.05,cost = 100).train(training, label, bands)

In [11]:
result = image.select(bands).classify(classifier)

In [40]:

## result with custom palette
palette = {
  'min':0,
  'max' :16,
  'bands' : ['classification'],
   'palette':['006400' ,'ffbb22', 'ffff4c', 'f096ff', 'fa0000', 'b4b4b4',
            'f0f0f0', '0064c8', '0096a0', '00cf75', 'fae6a0']
}

Map.addLayer(result, palette, 'classfied')
Map.centerObject(result)
Map

Map(center=[23.101347181130787, 78.0196924881727], controls=(WidgetControl(options=['position', 'transparent_b…

In [12]:


# result with random palette
Map.addLayer(result.randomVisualizer(), {}, 'classfied')
Map.centerObject(result)
Map

Map(bottom=28758.0, center=[23.02918734674459, 78.04412841796876], controls=(WidgetControl(options=['position'…

In [77]:

test = image.sampleRegions(**{
  'collection' : in_fc ,
  'properties' : [label],
  'scale': 30
})


def svm_function(svm_type,svm_cost):
    classifier = ee.Classifier.libsvm(svmType='C_SVC',kernelType = svm_type, cost = svm_cost)\
        .train(training, label, bands)
    return test\
    .classify(classifier) \
    .errorMatrix(label, 'classification') \
    .accuracy().getInfo()
    



In [104]:
svm_type = ['LINEAR','POLY','RBF']

svm_cost = [.1, 10, 100,1000]


import itertools
vals = list(itertools.product(svm_type, svm_cost))


In [105]:
vals

[('LINEAR', 0.1),
 ('LINEAR', 10),
 ('LINEAR', 100),
 ('LINEAR', 1000),
 ('POLY', 0.1),
 ('POLY', 10),
 ('POLY', 100),
 ('POLY', 1000),
 ('RBF', 0.1),
 ('RBF', 10),
 ('RBF', 100),
 ('RBF', 1000)]

In [106]:
cost_split_list = []
for i in range(len(vals)):
    val = vals[i][1]
    cost_split_list.append(val)
cost_split_list 

[0.1, 10, 100, 1000, 0.1, 10, 100, 1000, 0.1, 10, 100, 1000]

In [107]:
svm_type_list = []
for i in range(len(vals)):
    val = vals[i][0]
    svm_type_list.append(val)
svm_type_list

['LINEAR',
 'LINEAR',
 'LINEAR',
 'LINEAR',
 'POLY',
 'POLY',
 'POLY',
 'POLY',
 'RBF',
 'RBF',
 'RBF',
 'RBF']

In [108]:
accs = list(map(svm_function, svm_type_list,cost_split_list ))

In [90]:
cost_split_list

[0.1, 10, 100, 1000]

In [109]:
accs

[0.36180904522613067,
 0.5025125628140703,
 0.5728643216080402,
 0.5829145728643216,
 0.36180904522613067,
 0.36180904522613067,
 0.36180904522613067,
 0.36180904522613067,
 0.36180904522613067,
 0.36180904522613067,
 0.36180904522613067,
 0.4824120603015075]

## Valdiation

In [18]:

in_shp = './145044_20100428_roi/145044_20100428_200points.random_points2/145044_20100428_200points.shp'


In [19]:
in_fc = geemap.shp_to_ee(in_shp)


out_csv = ('./145044_20100428_models/svm/145044_20100428_200points_validation.csv')
geemap.extract_values_to_points(in_fc, result, out_csv)
validation_df = pd.read_csv(out_csv)

validation_df['class'].value_counts()

Generating URL ...
Please wait ...
Data downloaded to /Users/najah/work/internships/meghna/LT05_L1TP_145044_20100428_20161016_01_T1/145044_20100428_models/svm/145044_20100428_200points_validation.csv


11    73
5     44
1     17
4     16
13    13
2     13
15    10
16     6
17     5
8      3
Name: class, dtype: int64

In [44]:
validation_df['first'].value_counts()

11.0    122
5.0      68
1.0       6
8.0       3
Name: first, dtype: int64

In [45]:
cf = pd.crosstab( validation_df['class'],validation_df['first'], colnames=['Predicted'],rownames=['Actual'], margins=True).reindex(columns=[1,2,4,5,8,11,13,15,16],index=[1,2,4,5,8,11,13,15,16], fill_value=0)
cf

Predicted,1,2,4,5,8,11,13,15,16
Actual,,,,,,,,,
1,3,0,0,3,0,11,0,0,0
2,0,0,0,11,0,2,0,0,0
4,2,0,0,2,0,12,0,0,0
5,0,0,0,34,0,10,0,0,0
8,0,0,0,0,2,1,0,0,0
11,0,0,0,10,1,61,0,0,0
13,0,0,0,6,0,7,0,0,0
15,1,0,0,0,0,9,0,0,0
16,0,0,0,1,0,5,0,0,0


In [46]:
np.diag(cf).sum() / cf.to_numpy().sum()

0.5154639175257731